In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

# -----------------------------
# 1) Configuration
# -----------------------------
T = 120  # months (~10 years)
names = [f"Stock_{i+1}" for i in range(10)]

# target individual stock-bond correlations (mix of low/high, some negative)
rho_iB = np.array([ 0.05, 0.10, 0.15, 0.20, 0.25,
                    0.30, 0.05, -0.05, 0.12, 0.18 ])

# monthly volatilities (stocks higher than bond)
sigma_b = 0.03   # 3% monthly vol for "bond" total return
sigma_i = np.array([0.18, 0.20, 0.22, 0.19, 0.21,
                    0.23, 0.18, 0.20, 0.19, 0.22])

# cap weights for the stock index (sum to 1)
w = np.array([0.08, 0.12, 0.10, 0.07, 0.11,
              0.09, 0.06, 0.08, 0.13, 0.16])
w = w / w.sum()

# Common equity factor share to induce realistic stock–stock correlation
# alpha near 0 => stocks mostly idiosyncratic; near 1 => mostly common factor.
alpha = 0.6

# -----------------------------
# 2) Simulate factor shocks
# -----------------------------
# Standard normals for: bond (z_b), common equity factor (z_m), idiosyncratic (z_ei)
z_b = np.random.randn(T)
z_m = np.random.randn(T)
z_e = np.random.randn(T, len(names))

# Bond return series (centered, no drift for simplicity)
r_b = sigma_b * z_b  # 10Y "bond" total return proxy

# -----------------------------
# 3) Construct stock returns with target corr(stock_i, bond) = rho_iB
#    r_i = sigma_i * [ rho_iB * z_b  + sqrt(1 - rho_iB^2) * ( alpha*z_m + sqrt(1-alpha^2)*z_ei ) ]
#    -> Ensures corr with bond ≈ rho_iB and gives realistic stock-stock correlation via z_m
# -----------------------------
sqrt_1_r2 = np.sqrt(1 - rho_iB**2)
z_combo = alpha * z_m.reshape(-1, 1) + np.sqrt(1 - alpha**2) * z_e
stocks = sigma_i.reshape(1, -1) * (rho_iB.reshape(1, -1) * z_b.reshape(-1, 1) + sqrt_1_r2.reshape(1, -1) * z_combo)

df = pd.DataFrame(stocks, columns=names)
df.index = pd.date_range("2015-01-31", periods=T, freq="M")
df["Bond_10Y"] = r_b

# -----------------------------
# 4) Individual vs. aggregate correlations
# -----------------------------
# Individual stock-bond correlations (sample estimates)
indiv_corr = df[names].corrwith(df["Bond_10Y"])

# Portfolio (aggregate) stock index: cap-weighted
r_idx = df[names].values @ w
df["Stock_Index"] = r_idx

# Aggregate (index) correlation with bond (direct)
agg_corr_direct = np.corrcoef(r_idx, r_b)[0,1]

# -----------------------------
# 5) Correlation leverage math (Choueifaty–Coignard)
#    Portfolio vol, per-asset vol ratios, diversification ratio DR
# -----------------------------
# Sample vols
sig_i_hat = df[names].std().values           # stock vols
sig_p_hat = df["Stock_Index"].std()          # portfolio vol

# Volatility ratios phi_i = sigma_i / sigma_p
phi = sig_i_hat / sig_p_hat

# Diversification ratio DR = sum_i w_i * sigma_i / sigma_p = sum_i w_i * phi_i
DR = np.sum(w * phi)  # >= 1

# Rebalanced weights for the correlation formula: omega_i = (w_i * phi_i) / DR, sum omega_i = 1
omega = (w * phi) / DR

# Weighted average of individual corr, then "leveraged" by DR:
# Paper's relation: rho_{S,B} = DR * sum_i omega_i * rho_{i,B}
agg_corr_formula = DR * np.sum(omega * indiv_corr.values)

# -----------------------------
# 6) Diagnostics: "vol down, corr up" intuition
# -----------------------------
# Simple average of individual correlations (cap-weighted by w)
weighted_avg_indiv_corr = np.sum(w * indiv_corr.values)

results = {
    "Direct aggregate corr (Index vs Bond)": agg_corr_direct,
    "Formula aggregate corr (DR * sum omega_i * rho_iB)": agg_corr_formula,
    "Diversification Ratio (DR)": DR,
    "Index vol (sample)": sig_p_hat,
    "Cap-weighted avg indiv corr (sum w_i * rho_iB)": weighted_avg_indiv_corr
}

# -----------------------------
# 7) Tidy outputs + Excel export
# -----------------------------
summary = pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

per_stock = pd.DataFrame({
    "Weight": w,
    "Sigma_hat": sig_i_hat,
    "Phi_i = Sigma_i / Sigma_p": phi,
    "Omega_i": omega,
    "IndivCorr_iB (sample)": indiv_corr.values
}, index=names)

with pd.ExcelWriter("stock_bond_correlation_demo.xlsx", engine="xlsxwriter") as xw:
    df.to_excel(xw, sheet_name="Timeseries")
    per_stock.to_excel(xw, sheet_name="PerStock")
    summary.to_excel(xw, sheet_name="Summary")

print("=== SUMMARY ===")
print(summary, "\n")
print("Per-stock table (first few rows):")
print(per_stock.head(), "\n")
print("Saved Excel to: stock_bond_correlation_demo.xlsx")

=== SUMMARY ===
                                                       Value
Direct aggregate corr (Index vs Bond)               0.252666
Formula aggregate corr (DR * sum omega_i * rho_iB)  0.252666
Diversification Ratio (DR)                          1.462351
Index vol (sample)                                  0.141477
Cap-weighted avg indiv corr (sum w_i * rho_iB)      0.166927 

Per-stock table (first few rows):
         Weight  Sigma_hat  Phi_i = Sigma_i / Sigma_p   Omega_i  \
Stock_1    0.08   0.167379                   1.183083  0.064722   
Stock_2    0.12   0.216672                   1.531499  0.125674   
Stock_3    0.10   0.231463                   1.636045  0.111878   
Stock_4    0.07   0.185773                   1.313098  0.062856   
Stock_5    0.11   0.203435                   1.437940  0.108164   

         IndivCorr_iB (sample)  
Stock_1               0.133393  
Stock_2               0.141665  
Stock_3               0.249010  
Stock_4               0.170065  
Stock_5       

C:\Users\gigan\AppData\Local\Temp\ipykernel_18964\374613230.py:51: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df.index = pd.date_range("2015-01-31", periods=T, freq="M")
